### RunnableBranch

### 입력에 따른 동적 로직 라우팅

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("edu-CH13-LECL-Advanced")

LangSmith 추적을 시작합니다.
[프로젝트명]
edu-CH13-LECL-Advanced


간단한 예시

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """주어진 사용자 질문을 `수학`, `과학`, 또는 `기타` 중 하나로 분류하세요. 
한 단어 이상으로 응답하지 마세요.

<question>
{question}
</question>

Classification:"""
)

chain = (
    prompt
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

In [4]:
chain.invoke({"question": "2+2는 무엇입니까?"})

'수학'

In [5]:
chain.invoke({"question": "작용 반작용의 법칙은 무엇인가요?"})

'과학'

In [6]:
chain.invoke({"question": "Google은 어떤 회사인가요?"})

'기타'

In [7]:
math_chain = (
    PromptTemplate.from_template(
        """You are an expert in math. \
Always answer questions starting with "깨봉선생님께서 말씀하시기를..". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOpenAI(model="gpt-4o-mini")
)

science_chain = (
    PromptTemplate.from_template(
        """You are an expert in science. \
Always answer questions starting with "아이작 뉴턴 선생님께서 말씀하시기를..". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOpenAI(model="gpt-4o-mini")
)


general_chain = (
    PromptTemplate.from_template(
        """Respond to the following question concisely:

Question: {question}
Answer:"""
    )
    # OpenAI의 LLM을 사용합니다.
    | ChatOpenAI(model="gpt-4o-mini")
)

### 사용자 정의 함수 만들기 

In [8]:
def route(info):
    if "수학" in info["topic"].lower():
        return math_chain
    elif "과학" in info["topic"].lower():
        return math_chain
    else:
        return general_chain

In [10]:
from operator import itemgetter
from langchain_core.runnables import RunnableLambda

full_chain = (
    {"topic": chain, "question": itemgetter("question")}
    | RunnableLambda(
        route
    )
    | StrOutputParser()
)

In [11]:
full_chain.invoke({"question": "미적분의 개념에 대해 말씀해 주세요."})

'깨봉선생님께서 말씀하시기를, 미적분은 수학의 한 분야로, 주로 변화를 다루는 학문입니다. 미적분은 두 가지 주요 개념인 미분과 적분으로 나눌 수 있습니다.\n\n미분은 함수의 변화율을 다루며, 특정 순간의 기울기를 찾는 과정이라고 할 수 있습니다. 예를 들어, 물체의 속도를 구하는 데 사용되며, 이는 위치의 변화가 시간에 따라 어떻게 이루어지는지를 나타냅니다.\n\n적분은 주어진 함수의 면적을 구하는 과정으로, 미분의 역과정이라고 말씀하실 수 있습니다. 이는 주어진 변화량을 누적해서 전체적인 양을 계산하는 데 사용됩니다. 예를 들어, 물체가 이동한 거리나 에너지의 총합을 계산하는 데 유용합니다.\n\n미적분은 물리학, 경제학, 공학 등 다양한 분야에서 중요한 역할을 하며, 복잡한 문제를 해결하기 위한 강력한 도구입니다.'